In [1]:
import csv
import nltk
import copy
import scipy
import numpy as np
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

import time

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home2/_harsha_/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/_harsha_/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load the data
train_data_path = './Data/train.csv'
test_data_path = './Data/test.csv'

train_data = None
test_data = None
with open(train_data_path, 'r') as file:
    reader = csv.reader(file)
    train_data = list(reader)
    train_data = train_data[1:] # remove header
with open(test_data_path, 'r') as file:
    reader = csv.reader(file)
    test_data = list(reader)
    test_data = test_data[1:] # remove header

In [3]:
train_data_description = [row[1] for row in train_data]
train_data_tokenised = [nltk.word_tokenize(description) for description in train_data_description]
test_data_description = [row[1] for row in test_data]
test_data_tokenised = [nltk.word_tokenize(description) for description in test_data_description]

train_data_tokenised = [[word.split('\\') for word in description] for description in train_data_tokenised]
test_data_tokenised = [[word.split('\\') for word in description] for description in test_data_tokenised]

train_data_temp = copy.deepcopy(train_data_tokenised)
test_data_temp = copy.deepcopy(test_data_tokenised)
train_data_tokenised = []
test_data_tokenised = []
for description in train_data_temp:
    train_data_tokenised.append([])
    for words in description:
        for word in words:
            if word != '':
                train_data_tokenised[-1].append(word)
for description in test_data_temp:
    test_data_tokenised.append([])
    for words in description:
        for word in words:
            if word != '':
                test_data_tokenised[-1].append(word)

train_data_tokenised = [[word.lower() for word in description] for description in train_data_tokenised]
test_data_tokenised = [[word.lower() for word in description] for description in test_data_tokenised]

vocab = list(set([word for description in train_data_tokenised for word in description]))
vocab.append('<oov>')
vocab = sorted(vocab)
vocab_size = len(vocab)

word_index = {word: i for i, word in enumerate(vocab)}

In [ ]:
# find the lengths of each sentence
train_data_lengths = [len(description) for description in train_data_tokenised]
test_data_lengths = [len(description) for description in test_data_tokenised]

# find the length histogram
plt.hist(train_data_lengths, bins=100)
plt.xlabel('Length of Description')
plt.ylabel('Frequency')
plt.title('Histogram of Description Lengths')
plt.show()

# find the 90th percentile of the lengths
train_data_lengths = np.array(train_data_lengths)
train_data_lengths_90 = np.percentile(train_data_lengths, 90)
train_data_lengths_95 = np.percentile(train_data_lengths, 95)
train_data_lengths_99 = np.percentile(train_data_lengths, 99)
print('90th percentile of lengths:', train_data_lengths_90)
print('95th percentile of lengths:', train_data_lengths_95)
print('99th percentile of lengths:', train_data_lengths_99)

NameError: name 'train_data_tokenised' is not defined

In [ ]:
co_occurrence_matrix = scipy.sparse.lil_matrix((vocab_size, vocab_size))

window_size = 2
for description in tqdm.tqdm(train_data_tokenised):
    for i in range(len(description)):
        for j in range(i+1, min(i+window_size, len(description))):
            a,b = word_index[description[i]], word_index[description[j]]
            co_occurrence_matrix[a,b] += 1
            co_occurrence_matrix[b,a] += 1

co_occurrence_matrix = co_occurrence_matrix.tocsr()

u, s, vt = scipy.sparse.linalg.svds(co_occurrence_matrix.astype(np.float64), k=50)

embeddings = u

  0%|          | 0/120000 [00:00<?, ?it/s]

100%|██████████| 120000/120000 [00:52<00:00, 2279.89it/s]


In [ ]:
for i in range(len(test_data_tokenised)):
    for j in range(len(test_data_tokenised[i])):
        if test_data_tokenised[i][j] not in word_index:
            test_data_tokenised[i][j] = '<oov>'

train_data_x = []
for description in train_data_tokenised:
    train_data_x.append([embeddings[word_index[word]] for word in description])
test_data_x = []
for description in test_data_tokenised:
    test_data_x.append([embeddings[word_index[word]] for word in description])

train_data_x = nn.utils.rnn.pad_sequence([torch.tensor(description) for description in train_data_x], batch_first=True)
test_data_x = nn.utils.rnn.pad_sequence([torch.tensor(description) for description in test_data_x], batch_first=True)

/home2/_harsha_/miniconda3/envs/gaussian_splatting/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484801627/work/torch/csrc/utils/tensor_new.cpp:201.)
  del sys.path[0]


In [ ]:
print(train_data_x.shape)
print(test_data_x.shape)

torch.Size([120000, 245, 50])
torch.Size([7600, 160, 50])
